# Pre-workout

## Import 

In [ ]:
!pip3 install opencv-python-headless # questo funziona a differenze dell import classico

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:

from tensorflow.keras.preprocessing import image
import tensorflow as tf
import cv2 
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow import keras
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K

## Classes and functions

### Dataset Class

In [ ]:
class Dataset(object):
    def __init__(self, data_path):
        #path of the dataset
        self.data_path = data_path

        #class list
        self.data_classes = [directory for directory in os.listdir(data_path) if os.path.isdir(data_path+directory)]

        # init lists and dictionary
        self.images = []
        self.labels = []
        self.class_names = {}

        # for each class and for each image save the image and the label in the lists 
        for c, c_name in enumerate(self.data_classes):
            temp_path = os.path.join(self.data_path, c_name)
            temp_images = os.listdir(temp_path)
            self.class_names[c] = c_name

            for i in temp_images:
                img_tmp = os.path.join(temp_path, i)


                if img_tmp.endswith('.jpg') or img_tmp.endswith('.JPEG'):
                   # img = image.load_img(img_tmp, target_size=(224,224))
                    img = cv2.imread(img_tmp, 3)
                    model_image_size = (128, 128)
                    resized_image = cv2.resize(img, model_image_size, interpolation = cv2.INTER_CUBIC)
                    resized_image = resized_image.astype(np.float32) / 255.0
                    self.images.append(resized_image)
                    self.labels.append(c)
                    

        print('Loaded {:d} images from {:s} '.format(len(self.images), self.data_path))



    def num_classes(self):
        # returns number of classes of the dataset
        return len(self.data_classes)
    
    def get_dataset(self):
        return (list(zip(self.images, self.labels)), self.class_names)

    def generate(self):

        datagen = ImageDataGenerator(
            rotation_range=10, # rotation
            width_shift_range=0.2, # horizontal shift
            height_shift_range=0.2, # vertical shift
            zoom_range=0.2, # zoom
            horizontal_flip=True, # horizontal flip
            brightness_range=[0.2,1.2]
            ) # brightness

        train_generator = datagen.flow_from_directory(
                  directory=self.data_path,
                  target_size=(128, 128), # resize to this size
                  color_mode="rgb", # for coloured images
                  batch_size=32, # number of images to extract from folder for every batch
                  #class_mode="binary", # classes to predict
                  seed=420 # to make the result reproducible
                  )




### make pairs fun

In [ ]:
def get_random_image_idx_same_class(dataset, classe):

    label = 'formaggio'

    while label != classe:
        idx = np.random.choice(len(dataset))
        label = dataset[idx][1]
    
    return idx

def get_random_image_idx_different_class(dataset, classe):

    label = classe

    while label == classe:
        idx = np.random.choice(len(dataset))
        label = dataset[idx][1]
    
    return idx


def make_pairs(dataset):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairLabels = []
    imgs, labels = list(zip(*dataset))
    numClasses = len(np.unique(labels))

    for img, label in dataset:
        #current image 
        currentImage = img
        label = label 

        #positive image 
        pos_idx = get_random_image_idx_same_class(dataset, label)
        pos_img = imgs[pos_idx]

        pairImages.append([currentImage, pos_img])
        pairLabels.append([1])

        #negative imahe
        neg_idx = get_random_image_idx_different_class(dataset, label)
        neg_img = imgs[neg_idx]

        pairImages.append([currentImage, neg_img])
        pairLabels.append([0])

    return (np.array(pairImages), np.array(pairLabels))

In [ ]:
def make_pairs_efficient(dataset):
     # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairLabels = []

    for idx, (img, label) in enumerate(dataset):
  
        
        #positive image 
        pos_idx = get_random_image_idx_same_class(dataset, label)
        

        pairImages.append([idx, pos_idx])
        pairLabels.append([1])

        #negative imahe
        neg_idx = get_random_image_idx_different_class(dataset, label)
        

        pairImages.append([idx, neg_idx])
        pairLabels.append([0])

    return (np.array(pairImages), np.array(pairLabels))

# Dataset

training is a list of tuple (image, label),  class_names is a dictionary : class_names[3] = struzzo

In [ ]:
#training_path = '/datasets/animali/animals_dataset_the_ostriches/unbalanced_dataset_2304/training/'
validation_path = '/work/dataset/validation/gallery/'
training_path = '/work/dataset/training/'

training, class_names = Dataset(data_path = training_path).get_dataset()
validation, class_names_val = Dataset(data_path = validation_path).get_dataset()


Loaded 2174 images from /work/dataset/training/ 
Loaded 463 images from /work/dataset/validation/gallery/ 


pairTrain is a tuple with 2 images, labelTrain is 0 is the two images belong to 2 different classes 1 if are the same animal

In [ ]:

pairTrain, labelTrain = make_pairs_efficient(training)
pairTest, labelTest = make_pairs_efficient(validation)


## Explore Dataset

# Model

In [ ]:
def build_siamese_model_old(input_shape, embeddingDim=48):

    # specify the inputs for the feature extractor network
    inputs = Input(input_shape)

    # define the first set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (2, 2), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    # second set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    # prepare the final outputs
    pooledOutput = GlobalAveragePooling2D()(x)
    outputs = Dense(embeddingDim)(pooledOutput)
    # build the model
    model = Model(inputs, outputs)
    # return the model to the calling function
    return model


In [ ]:
def build_siamese_model(input_shape, embeddingDim):
    

    inputs = keras.layers.Input(input_shape)
    x = keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    x = keras.layers.Conv2D(64, 3, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(128, 3, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D((4, 4))(x)
    x = keras.layers.Conv2D(256, 3, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(256, 3, activation="relu")(x)
    x = keras.layers.GlobalMaxPool2D()(x)
    outputs = tfsim.layers.MetricEmbedding(embeddingDim)(x)

    # building model
    model = tfsim.models.SimilarityModel(inputs, outputs)
    model.summary()

    return model

In [ ]:
def eucledian_distance(vectors):

    (fa, fb) = vectors

    sum_squared = K.sum(K.square(fa-fb), axis= 1, keepdims = True)

    return K.sqrt(K.maximum(sum_squared, K.epsilon()))


In [ ]:
f_ext = build_siamese_model((128,128,3))
imgA = Input(shape=(128,128,3)) # non so cosa faccia sta cosa
imgB = Input(shape=(128,128,3))

featsA = f_ext(imgA)
featsB = f_ext(imgB)

In [ ]:
distance = Lambda(eucledian_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

In [ ]:
tr_imgs, tr_labels = zip(*training)
val_imgs, tr_labels = zip(*validation)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
imgs1 = np.array([tr_imgs[index] for index in pairTrain[:, 0] ])
imgs2 = np.array([tr_imgs[index] for index in pairTrain[:, 1] ])

val1 = [val_imgs[index] for index in pairTest[:, 0] ]
val2 = [val_imgs[index] for index in pairTest[:, 1] ]

datagen = ImageDataGenerator(
        rotation_range=10, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.2,1.2]) # brightness


model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

history = model.fit_generator(
	datagen.flow([imgs1, imgs2], labelTrain[:], batch_size=32),
	#validation_data=([val1, val2], labelTest[:]), 
	epochs=10)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
Epoch 1/10
136/136 [==============================] - 449s 3s/step - loss: 1.6201 - accuracy: 0.5030
Epoch 2/10
136/136 [==============================] - 444s 3s/step - loss: 0.6978 - accuracy: 0.5025
Epoch 3/10
136/136 [==============================] - 451s 3s/step - loss: 0.6953 - accuracy: 0.4892
Epoch 4/10
136/136 [==============================] - 423s 3s/step - loss: 0.6941 - accuracy: 0.5009
Epoch 5/10
136/136 [==============================] - 329s 2s/step - loss: 0.6939 - accuracy: 0.4922
Epoch 6/10
 90/136 [==================>...........] - ETA: 1:52 - loss: 0.6942 - accuracy: 0.5021

KeyboardInterrupt: 

In [ ]:


preds = model.predict([np.expand_dims(val_imgs[11], axis=0), np.expand_dims(val_imgs[11], axis=0)])
proba = preds[0][0]

proba

0.4775884

In [ ]:
proba

0.4775884

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 48)           20400       ['input_2[0][0]',                
                                                                  'input_3[0][0]']          

In [ ]:
len(training  )

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=93aceac2-8452-469e-8b02-c16d0438aa9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>